## Workflow for transcribing audio files with whisper.

#### Create the filename

In [2]:
import os
import datetime
import subprocess

#working_folder = "/Users/reblocke/audio_transcripts"
working_folder = "/Users/blocke/audio_transcripts"
os.makedirs(working_folder, exist_ok=True)

timestamp = datetime.datetime.now().strftime("transcription_%y-%m-%d_at_%H-%M.flac")
audio_output_path = os.path.join(working_folder, timestamp)
text_output_path = os.path.splitext(audio_output_path)[0] + ".txt"

print(audio_output_path)
print(text_output_path)

/Users/blocke/audio_transcripts/transcription_25-05-21_at_06-47.flac
/Users/blocke/audio_transcripts/transcription_25-05-21_at_06-47.txt


#### Command to record the transcription

Note, this can be done with the following terminal command:

!ffmpeg -f avfoundation -i ":2" -ac 1 -ar 16000 -b:a 192k output.wav   

In [4]:
# Start ffmpeg and enable writing to its stdin.
process = subprocess.Popen(
    ['ffmpeg', '-f', 'avfoundation', '-i', ':2', '-ac', '1', '-ar', '16000', '-b:a', '192k', audio_output_path],
    stdin=subprocess.PIPE
)

#### Run the below box to stop the recording

In [5]:
process.communicate(input=b'q')

size=   22528kB time=00:14:54.14 bitrate= 206.4kbits/s speed=   1x    

[q] command received. Exiting.

[out#0/flac @ 0x132f1faf0] video:0kB audio:22760kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.035562%
size=   22768kB time=00:14:54.33 bitrate= 208.5kbits/s speed=   1x    


(None, None)

#### Command to transcribe the audio just recorded

In [7]:
import whisper

model = whisper.load_model("turbo")
result = model.transcribe(audio_output_path, fp16=False)
print(result["text"])
with open(text_output_path, "w", encoding="utf-8") as f:
    f.write(result["text"])

AttributeError: module 'whisper' has no attribute 'load_model'

#### Delete the output_path file

In [ ]:
os.remove(audio_output_path)